In [1]:
%pip install pandas plotly matplotlib
%pip install -U nbformat

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
import json
import pandas as pd
import random
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [4]:
# UPDATE_PARTICIPANT_TO_ACTION,    // ACTION 참여자 수정(참여자 추가 및 삭제) 이벤트
# CREATE_PROJECT_PROGRESS_ACTION,  // ACTION 생성 이벤트
# DELETE_PROJECT_PROGRESS_ACTION,  // ACTION 삭제 이벤트
# DONE_PROJECT_PROGRESS_ACTION     // ACTION 상태 완료로 변경이벤트

In [5]:
LOG_DATA_PATH = "data\\final_user-actions_dummy.json"

# 중첩 구조 평탄화해서 읽기
with open(LOG_DATA_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)
    # data = [json.loads(line.strip()) for line in f if line.strip()]  # .jsonl일 때 읽어오는 방법
    
df = pd.json_normalize(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   event                 100 non-null    object
 1   userId                100 non-null    object
 2   username              100 non-null    object
 3   timestamp             100 non-null    object
 4   workspaceId           100 non-null    int64 
 5   details.participants  100 non-null    object
 6   details.state         100 non-null    object
 7   details.name          100 non-null    object
 8   details.importance    100 non-null    int64 
 9   details.startDate     47 non-null     object
 10  details.endDate       47 non-null     object
dtypes: int64(2), object(9)
memory usage: 8.7+ KB


In [6]:
df

,event,userId,username,timestamp,workspaceId,details.participants,details.state,details.name,details.importance,details.startDate,details.endDate
0,CREATE_PROJECT_PROGRESS_ACTION,3,testuser3,2025-06-09T23:46:31.072466Z,1,"[{'userId': 3, 'username': 'testuser3'}, {'use...",IN_PROGRESS,배송 상태 추적,1,None,None
1,DONE_PROJECT_PROGRESS_ACTION,4,testuser4,2025-06-12T06:09:44.072507Z,1,"[{'userId': 1, 'username': 'testuser1'}, {'use...",DONE,배송비 계산,1,2025-06-12T03:09:44.072507,2025-06-12T06:09:44.072507
2,DONE_PROJECT_PROGRESS_ACTION,1,testuser1,2025-06-10T06:09:44.072540Z,1,"[{'userId': 1, 'username': 'testuser1'}, {'use...",DONE,결제 게이트웨이 연동,2,2025-06-07T06:09:44.072540,2025-06-10T06:09:44.072540
3,CREATE_PROJECT_PROGRESS_ACTION,2,testuser2,2025-05-29T07:23:58.072562Z,1,"[{'userId': 4, 'username': 'testuser4'}, {'use...",BEFORE,상품 리뷰 표시,3,None,None
4,CREATE_PROJECT_PROGRESS_ACTION,3,testuser3,2025-06-15T00:53:50.072582Z,1,"[{'userId': 2, 'username': 'testuser2'}, {'use...",BEFORE,알림 통계,5,None,None
...,...,...,...,...,...,...,...,...,...,...,...
95,CREATE_PROJECT_PROGRESS_ACTION,1,testuser1,2025-05-17T18:59:07.074155Z,1,"[{'userId': 5, 'username': 'testuser5'}, {'use...",IN_PROGRESS,정렬 기능,5,None,None
96,DONE_PROJECT_PROGRESS_ACTION,2,testuser2,2025-06-12T06:09:44.074168Z,1,"[{'userId': 4, 'username': 'testuser4'}, {'use...",DONE,KPI 모니터링,5,2025-06-07T06:09:44.074168,2025-06-12T06:09:44.074168
97,CREATE_PROJECT_PROGRESS_ACTION,3,testuser3,2025-05-31T22:31:09.074183Z,1,"[{'userId': 1, 'username': 'testuser1'}, {'use...",IN_PROGRESS,성능 지표,1,None,None
98,DONE_PROJECT_PROGRESS_ACTION,1,testuser1,2025-06-10T06:09:44.074195Z,1,"[{'userId': 2, 'username': 'testuser2'}, {'use...",DONE,실시간 통계,2,2025-06-04T06:09:44.074195,2025-06-10T06:09:44.074195


In [7]:
# unique() 체크
print("df['event'].unique(): ", df['event'].unique())
print("df['userId'].unique(): ", df['userId'].unique())
print("df['workspaceId'].unique(): ", df['workspaceId'].unique())

print("df['details.state'].unique(): ", df['details.state'].unique())
print("df['details.importance'].unique(): ", df['details.importance'].unique())

df['event'].unique():  ['CREATE_PROJECT_PROGRESS_ACTION' 'DONE_PROJECT_PROGRESS_ACTION']
df['userId'].unique():  ['3' '4' '1' '2' '5']
df['workspaceId'].unique():  [1]
df['details.state'].unique():  ['IN_PROGRESS' 'DONE' 'BEFORE']
df['details.importance'].unique():  [1 2 3 5 4]


In [8]:
# # 유저별 df 분류
df_user1 = df[df['userId'] == '1']
df_user2 = df[df['userId'] == '2']
df_user3 = df[df['userId'] == '3']
df_user4 = df[df['userId'] == '4']
df_user5 = df[df['userId'] == '5']

In [9]:
# 유저별 할당된 actions 갯수 확인
print(len(df_user1))
print(len(df_user2))
print(len(df_user3))
print(len(df_user4))
print(len(df_user5))

44
27
14
7
8


In [10]:
# 1. 담당자 불균형 분석 -> Bar plot
# - point columns = details.participants, details.importance
# - 작업이 특정 개발자에게 몰리는가 -> userId별 분류 -> status별 분류 -> 시각화
#   - status는 왜? 이미 진행 중인 작업이 많은 개발자에게 얼마나 더 할당되어 있는가
# - 각 개발자가 맡은 업무의 중요도 분포는 어떻게 되는가
# - status는 중요하지 않음, participants 체크가 중요

In [11]:
grouped1 = df_user1.groupby(['userId', 'details.state', 'details.importance']).size().reset_index(name='count')
grouped2 = df_user2.groupby(['userId', 'details.state', 'details.importance']).size().reset_index(name='count')
grouped3 = df_user3.groupby(['userId', 'details.state', 'details.importance']).size().reset_index(name='count')
grouped4 = df_user4.groupby(['userId', 'details.state', 'details.importance']).size().reset_index(name='count')
grouped5 = df_user5.groupby(['userId', 'details.state', 'details.importance']).size().reset_index(name='count')

In [12]:
grouped1

,userId,details.state,details.importance,count
0,1,BEFORE,1,3
1,1,BEFORE,2,5
2,1,BEFORE,3,7
3,1,BEFORE,4,3
4,1,BEFORE,5,5
5,1,DONE,1,2
6,1,DONE,2,8
7,1,DONE,3,2
8,1,DONE,4,3
9,1,DONE,5,4


In [13]:
fig = px.bar(
    grouped1,
    x='details.state',
    y='count',
    color='details.importance',
    barmode='stack',
    title='담당자 불균형 분석'
)
fig.show()


In [14]:
json_temp = grouped1.to_dict(orient='records')

with open(f"data\\stat1-user1.json", "w", encoding="utf-8") as f:
    json.dump(json_temp, f, ensure_ascii=False, indent=2)